In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import glob
from skimage.io import imread
import numpy as np
import pandas as pd

In [ ]:
#! pip install tensorflow-gpu==1.10.0

In [6]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8863684134146818704
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 17047136807778442531
physical_device_desc: "device: XLA_CPU device"
]


In [8]:
class dataframe():
    def __init__(self):
        self.Path='/content/drive/MyDrive/teeth/crop_rotated_1-5/*.png'
        self.csv_path='/content/drive/MyDrive/teeth/train/nor_img_2020.csv'
        self.image=[]
        self.Label = []
        self.Label2=[]
        self.comb=[]
    def crop_image(self):
        img_path=sorted(glob.glob(self.Path))
        for i in range(len(img_path)):
            a = imread(img_path[i])
            m,n=a.shape
            b=np.zeros((m,n,3))
            for j in range(0,3):
                b[:,:,j]=a
            self.image.append(b)
        return self.image
    def lab(self):
        ori1 = pd.read_csv(self.csv_path) 
        title =list(ori1.columns[:])
        
        for i in range(len(ori1[title[2]])):
            for j in range(0,6):
                if ori1[title[0]][i]==0:
                    print(ori1[title[2]][i]+"_T1")
                else:
                    self.Label.append(ori1[title[0]][i])
            for j in range(0,6):
                if ori1[title[1]][i]==0:
                    print(ori1[title[2]][i]+"_T2")
                else:
                    self.Label.append(ori1[title[1]][i])
        return self.Label
    def lab2(self):
        for j in range(len(self.Label)):
            if self.Label[j] ==1:
                self.Label2.append([1,0,0,0,0])
            if self.Label[j] ==2:
                self.Label2.append([0,1,0,0,0])
            if self.Label[j] ==3:
                self.Label2.append([0,0,1,0,0])
            if self.Label[j] ==4:
                self.Label2.append([0,0,0,1,0])
            if self.Label[j] ==5:
                self.Label2.append([0,0,0,0,1])
        return self.Label2
    def combine(self):
        
        for i in range(len(self.image)):
            two=[self.image[i],self.Label2[i]]
            self.comb.append(two)
        return self.comb

In [9]:
if __name__ == '__main__':
    Data=dataframe()
    Data.crop_image()
    Data.lab()
    Data.lab2()
    Data.combine()

P046_T2
P046_T2
P046_T2
P046_T2
P046_T2
P046_T2
P058_T1
P058_T1
P058_T1
P058_T1
P058_T1
P058_T1
P058_T2
P058_T2
P058_T2
P058_T2
P058_T2
P058_T2
P061_T1
P061_T1
P061_T1
P061_T1
P061_T1
P061_T1
P061_T2
P061_T2
P061_T2
P061_T2
P061_T2
P061_T2
P075_T2
P075_T2
P075_T2
P075_T2
P075_T2
P075_T2
P088_T2
P088_T2
P088_T2
P088_T2
P088_T2
P088_T2


In [10]:
from sklearn.model_selection import train_test_split
import random
from tensorflow.keras.callbacks import ModelCheckpoint , EarlyStopping , CSVLogger ,ReduceLROnPlateau ,History
from tensorflow.keras.optimizers import Adam,SGD

In [11]:
train_data, vali_data = train_test_split(Data.comb,random_state=0,shuffle=True,train_size=0.7)

In [12]:
def data_gan_fn(  data, batch_size  ):
    data = random.sample( data , len(data) )
    j = 0
    img_set1=[]
    Lab=[]
    while True:
        batch_start = j*batch_size
        if ( batch_start + batch_size ) > len( data ):
            img_set1=[]
            Lab=[]
            batch_start2 = 0
            batch_end = len( data )
            batch_end2 = batch_start + batch_size - len( data )
            out_set = data[ batch_start : batch_end ] + data[ batch_start2 : batch_end2 ]
            for i in range(len(out_set)):
                img_set1.append(out_set[i][0])
                Lab.append(out_set[i][1])
            img_set1=np.stack(img_set1)
            Lab=np.stack(Lab)
            yield img_set1,Lab
        else :
            img_set1=[]
            Lab=[]
            out_set = data[ batch_start : batch_start + batch_size ]
            for i in range(len(out_set)):
                img_set1.append(out_set[i][0])
                Lab.append(out_set[i][1])
            img_set1=np.stack(img_set1)
            Lab=np.stack(Lab)
            yield img_set1,Lab
        j = j + 1
        if (  j*batch_size )  >=  len( data ):
            j = 0
            img_set1=[]
            Lab=[]

In [13]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense,Dropout,Conv2D,MaxPooling2D,Flatten
from tensorflow.keras.models import Model

In [38]:
def create_model():
  model = VGG16(weights='imagenet', include_top=True)
  model.layers.pop()
  x = model.layers[-1].output
  x=Dense(5, activation='relu')(x)
  x=Dense(5, activation='softmax')(x)
  # 重新建立模型結構
  model=Model(model.input,x)
  model.summary()
  return model

#以下是gpu的使用方法

In [ ]:
!export GCSFUSE_REPO=gcsfuse-`lsb_release -c -s`
!echo "deb http://packages.cloud.google.com/apt $GCSFUSE_REPO main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

!sudo apt-get update
!sudo apt-get install gcsfuse

In [32]:
import os
import pprint
import tensorflow as tf

if 'COLAB_TPU_ADDR' not in os.environ:
    print('ERROR: Not connected to a TPU runtime')
else:
    tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    print ('TPU address is', tpu_address)

TPU address is grpc://10.107.76.242:8470


In [83]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
#tpu=tf.distribute.TPUStrategy()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)
with strategy.scope():
  model = create_model()
  model.compile( optimizer = Adam( lr = 1e-6 ) , 
              loss = 'categorical_crossentropy' , 
              metrics=['accuracy'] )

INFO:tensorflow:Initializing the TPU system: grpc://10.107.76.242:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.107.76.242:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

開始訓練

In [85]:
epoch = 50000000
batch_size = 4
data_gen  = data_gan_fn(train_data ,batch_size )
val_data = data_gan_fn(vali_data ,batch_size )
steps_per_epoch = int( np.ceil( len( train_data )/batch_size ) )
validation_steps = int( np.ceil( len(vali_data )/batch_size ) )
model_path =   '/content/drive/MyDrive/teeth/saved_models/NTP.h5' 
check_point = ModelCheckpoint( model_path,monitor='val_loss' , 
                              save_best_only= True , verbose = 2)
csv_logger = CSVLogger( '/content/drive/MyDrive/teeth/logs/NTP.log '  )
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, 
#                   patience=40, min_delta=0.0001)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=50, min_lr=0.0001)

In [80]:
x_train=[]
y_train=[]
x_vail=[]
y_vail=[]
for i in range(len(train_data)):
  x_train.append(train_data[i][0])
  y_train.append(train_data[i][1])
x_train=np.stack(x_train)
y_train=np.stack(y_train)
for i in range(len(vali_data)):
  x_vail.append(vali_data[i][0])
  y_vail.append(vali_data[i][1])
x_vail=np.stack(x_vail)
y_vail=np.stack(y_vail)

In [ ]:
#model.fit(
#  x=(x_train),y=(y_train),
#  epochs=5,
#  steps_per_epoch=60,
#  validation_data=((x_vail),(y_vail)),
#  )

In [ ]:
model.fit( x=(x_train),y=(y_train), 
          epochs=epoch , 
          verbose=True , 
          steps_per_epoch=steps_per_epoch  , 
          callbacks=[ check_point , csv_logger , reduce_lr ] , 
          validation_data=((x_vail),(y_vail)),
          validation_steps = validation_steps)

Epoch 1/50000000
  2/157 [..............................] - ETA: 6s - loss: 1.6093 - accuracy: 0.1250    WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0034s vs `on_train_batch_end` time: 0.0711s). Check your callbacks.


157/157 [==============================] - ETA: 0s - loss: 1.6094 - accuracy: 0.2304WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0020s vs `on_test_batch_end` time: 0.0192s). Check your callbacks.


串流輸出內容已截斷至最後 5000 行。
Epoch 00222: val_loss improved from 1.58777 to 1.58772, saving model to /content/drive/MyDrive/teeth/saved_models/NTP.h5
157/157 [==============================] - 26s 169ms/step - loss: 1.5501 - accuracy: 0.5952 - val_loss: 1.5877 - val_accuracy: 0.3234
Epoch 223/50000000
157/157 [==============================] - ETA: 0s - loss: 1.5499 - accuracy: 0.5952
Epoch 00223: val_loss improved from 1.58772 to 1.58760, saving model to /content/drive/MyDrive/teeth/saved_models/NTP.h5
157/157 [==============================] - 26s 166ms/step - loss: 1.5499 - accuracy: 0.5952 - val_loss: 1.5876 - val_accuracy: 0.3160
Epoch 224/50000000
157/157 [==============================] - ETA: 0s - loss: 1.5511 - accuracy: 0.5808
Epoch 00224: val_loss did not improve from 1.58760
157/157 [==============================] - 18s 116ms/step - loss: 1.5511 - accuracy: 0.5808 - val_loss: 1.5946 - val_accuracy: 0.2751
Epoch 225/50000000
157/157 [==============================] - ETA: 0s - loss

In [ ]:
#result = model.fit_generator( data_gen , epochs=epoch , verbose=True , 
#                              steps_per_epoch=steps_per_epoch  , 
#                              callbacks=[ check_point , csv_logger , reduce_lr ] , 
#                              validation_data = val_data,validation_steps = validation_steps)

In [ ]:
#! pip install tensorflow-gpu==2.0.0